## Parking Controller via MPC with Obstacles

###System dynamics:
$z = [x,y,\phi]$, $u = [d_s, R, \beta]$
Inputs are: 
$d_s$ is the traveled straight  distance,
$R$ is the turning radius (after $d_s$)
$\beta$ is the traveled angle with turning radius $R$


###Optimization Algorithm

In [82]:
#Pkg.add("JuMP"); Pkg.add("Ipopt"); Pkg.add("Gadfly"); Pkg.add("Interact")
#Pkg.checkout("JuMP") # need latest master for matrices
using JuMP, Ipopt
using PyPlot
using PyCall

function solveMPC(car_size,n,m,T,z0,zT,zmin,zmax,umin,umax,obs,side)
    l=car_size[1]
    dr=car_size[2]
    w=car_size[3]
    mpc = Model(solver=IpoptSolver(print_level=3))
    @defVar(mpc,  zmin[i] <= z[i=1:n,t=0:T] <= zmax[i])
    @defVar(mpc, -umax[i] <= u[i=1:m,t=0:T-1] <= umax[i])
    # Cost
    @setNLObjective(mpc, Min,sum{u[1,t]^2+(u[3,t]*u[2,t])^2,t=0:T-1})
    
    # Link state and control across the horizon
    for t = 0:T-1        
        @addNLConstraint(mpc, z[1,t+1] == z[1,t] + u[1,t]*cos(z[3,t])-u[2,t]*sin(z[3,t])+u[2,t]*sin(z[3,t]+u[3,t]))
        @addNLConstraint(mpc, z[2,t+1] == z[2,t] + u[1,t]*sin(z[3,t])+u[2,t]*cos(z[3,t])-u[2,t]*cos(z[3,t]+u[3,t]))
        @addNLConstraint(mpc, z[3,t+1] == z[3,t] + u[3,t])
    end

    # Obstacle Avoidace constraints coded as disjunction
    if 0<1
        #Increase resolution in space for obstacle avoidance cehcking. For ever segment use rep points
        rep=5;
        @defVar(mpc,  zmin[i] <= zs[i=1:n,t=0:2*rep*T] <= zmax[i])
        @addConstraint(mpc, zs[:,0] .== z0)
        t=0
        for k = 0:T-1        
            for j = 1:rep        
                t=t+1
                # Staight segment
                @addNLConstraint(mpc, zs[1,t] == zs[1,t-1] + 1/rep*u[1,k]*cos(zs[3,t-1]))
                @addNLConstraint(mpc, zs[2,t] == zs[2,t-1] + 1/rep*u[1,k]*sin(zs[3,t-1]))
                @addNLConstraint(mpc, zs[3,t] == zs[3,t-1] )
            end
            for j = 1:rep        
                t=t+1
                # Curved segment
                @addNLConstraint(mpc, zs[1,t] == zs[1,t-1] -u[2,k]*sin(zs[3,t-1])+u[2,k]*sin(zs[3,t-1]+1/rep*u[3,k]))
                @addNLConstraint(mpc, zs[2,t] == zs[2,t-1] +u[2,k]*cos(zs[3,t-1])-u[2,k]*cos(zs[3,t-1]+1/rep*u[3,k]))
                @addNLConstraint(mpc, zs[3,t] == zs[3,t-1] + 1/rep*u[3,k])
            end
        end
      
        # Obstacle Avoidace constraints coded as disjunction
        @defVar(mpc, lambda[i=1:16,t=0:2*rep*T] >= 0)                     
        for t = 0:2*rep*T
            # Four corners of the car
            #A=[x+l*cos(psi)+w*sin(psi),y+l*sin(psi)-w*cos(psi)]
            #B=[x+w*sin(psi),y-w*cos(psi)]
            #C=[x+l*cos(psi)-w*sin(psi),y+l*sin(psi)+w*cos(psi)]
            #D=[x-w*sin(psi),y+w*cos(psi)]

            #To be parametrized later : parallel parking bay with two obstacles
            # x>=xl OR y-yt>=0 
            # x<=xr OR y-yt>=0 
            # Evaluated at A,B,C,D
            xl=obs[1]
            xr=obs[2]
            yt=obs[3]

            @addNLConstraint(mpc, lambda[1,t]*(zs[1,t]+l*cos(zs[3,t])+w*sin(zs[3,t])-xl)+lambda[2,t]*(zs[2,t]+l*sin(zs[3,t])-w*cos(zs[3,t])-yt) >=0)
            @addConstraint(mpc, lambda[1,t]+lambda[2,t] == 1)
            @addNLConstraint(mpc, lambda[3,t]*(zs[1,t]-dr*cos(zs[3,t])+w*sin(zs[3,t])-xl)+lambda[4,t]*(zs[2,t]-dr*sin(zs[3,t])-w*cos(zs[3,t])-yt) >=0)
            @addConstraint(mpc, lambda[3,t]+lambda[4,t] == 1)
            @addNLConstraint(mpc, lambda[5,t]*(zs[1,t]+l*cos(zs[3,t])-w*sin(zs[3,t])-xl)+lambda[6,t]*(zs[2,t]+l*sin(zs[3,t])+w*cos(zs[3,t])-yt) >=0)
            @addConstraint(mpc, lambda[5,t]+lambda[6,t] == 1)
            @addNLConstraint(mpc, lambda[7,t]*(zs[1,t]-dr*cos(zs[3,t])-w*sin(zs[3,t])-xl)+lambda[8,t]*(zs[2,t]-dr*sin(zs[3,t])+w*cos(zs[3,t])-yt) >=0)
            @addConstraint(mpc, lambda[7,t]+lambda[8,t] == 1)

            @addNLConstraint(mpc, lambda[9,t]*(-(zs[1,t]+l*cos(zs[3,t])+w*sin(zs[3,t]))+xr)+lambda[10,t]*(zs[2,t]+l*sin(zs[3,t])-w*cos(zs[3,t])-yt) >=0)
            @addConstraint(mpc, lambda[9,t]+lambda[10,t] == 1)
            @addNLConstraint(mpc, lambda[11,t]*(-(zs[1,t]-dr*cos(zs[3,t])+w*sin(zs[3,t]))+xr)+lambda[12,t]*(zs[2,t]-dr*sin(zs[3,t])-w*cos(zs[3,t])-yt) >=0)
            @addConstraint(mpc, lambda[11,t]+lambda[12,t] == 1)
            @addNLConstraint(mpc, lambda[13,t]*(-(zs[1,t]+l*cos(zs[3,t])-w*sin(zs[3,t]))+xr)+lambda[14,t]*(zs[2,t]+l*sin(zs[3,t])+w*cos(zs[3,t])-yt) >=0)
            @addConstraint(mpc, lambda[13,t]+lambda[14,t] == 1)
            @addNLConstraint(mpc, lambda[15,t]*(-(zs[1,t]-dr*cos(zs[3,t])-w*sin(zs[3,t]))+xr)+lambda[16,t]*(zs[2,t]-dr*sin(zs[3,t])+w*cos(zs[3,t])-yt) >=0)
            @addConstraint(mpc, lambda[15,t]+lambda[16,t] == 1)
            
            
            @addNLConstraint(mpc, zs[2,t]+l*sin(zs[3,t])-w*cos(zs[3,t]) >=zmin[2])
            @addNLConstraint(mpc, zs[2,t]-dr*sin(zs[3,t])-w*cos(zs[3,t])>=zmin[2])
            @addNLConstraint(mpc, zs[2,t]+l*sin(zs[3,t])+w*cos(zs[3,t]) >=zmin[2])
            @addNLConstraint(mpc, zs[2,t]-dr*sin(zs[3,t])+w*cos(zs[3,t])>=zmin[2])
 

        end
    end
    
    for t = 0:T-1 
        if side[t+1]=='1'
           #Postitive Curvature Radius
           @addConstraint(mpc, u[2,t] >= umin[2])
           #print("added +,")
        else
            #Negative Curvature Radius
           @addConstraint(mpc, u[2,t] <= -umin[2])
           #print("added -,")
        end
    end

    # Initial conditions
    @addConstraint(mpc, z[:,0] .== z0)
    # Final state
    @addConstraint(mpc, z[:,T] .== zT)
    # Solve the NLP
    status= solve(mpc)
    return getValue(u), getValue(z), getObjectiveValue(mpc), status
end

solveMPC (generic function with 2 methods)

### Run Simulations

In [ ]:
#Car Parameters
n=3
m=3
lr  = 1.738
dr = 1.2
lf = 1.105
df = 1.2
width=0.93
#delta_max=25*pi/180
#  according to julian 
delta_max=48.2*pi/180

#paramters not to touch
L  = lr+lf+df
car_size=[L,dr,width]


# Constraints
zmin=[-20; 0;-8*pi]
umax=[30;10000;8*pi]
umin=[-umax[1];(lf+lr)/tan(delta_max);-umax[3]]


# Initial consitions, terminal condition and parkyng bay size

# EXAMPLE F - Exit parking --4 steps not feasible
z0 = [0;L+0.5;-pi/2+pi/20]  
zT = [0;8.0-lr;-pi]    
obs=[-1.5;2.5;lf+lr+df+dr+1]
zmax=[20;lf+lr+df+dr+3;8*pi]
Tmpc = 5 # MPC horizon


#  EXAMPLE A
zmax=[20;20;8*pi]
z0 = [-10;10;0]  
zT = [0;1.8;0]  
obs=[-3-dr;L+2;3]
Tmpc = 3 # MPC horizon

# EXAMPLE B
zmax=[20;20;8*pi]
z0 = [-10;10;0]  
zT = [0;width+0.3;0]  
obs=[-1-dr;L+2;2*width+0.3]
Tmpc = 4 # MPC horizon

# EXAMPLE C
zmax=[20;20;8*pi]
z0 = [-10;10;0]  
zT = [0;width+0.3;0]  
obs=[-1-dr;L+1;2*width+0.3]
Tmpc = 4 # MPC horizon

# EXAMPLE D
zmax=[20;20;8*pi]
z0 = [-10;2*width+0.3;0]  
zT = [0;width+0.3;0]  
obs=[-1-dr;L+1;2*width+0.3]
Tmpc = 4 # MPC horizon

# EXAMPLE E - Vertical parking form left 
zmax=[20;20;8*pi]
z0 = [-10;10;0]  
zT = [0;1.8;pi/2]  
obs=[-width-0.2;width+0.2;L+dr]
Tmpc = 2 # MPC horizon


# EXAMPLE F - Vertical parking from roght 9interesting, omre diccult than roght.. (cannot do tight)
zmax=[20;20;8*pi]
z0 = [-10;10;0]  
zT = [0;1.8;pi/2]  
obs=[-width-0.3;width+0.3;L+dr]
Tmpc = 2 # MPC horizon




bestcost=100000000
usol= zeros(n,Tmpc)
zsol = zeros(m,Tmpc)
# MPC
for j=0:2^Tmpc-1
    # check all positive or negative combinatiosn of curvature radius
    side=bin(j,Tmpc)
    #solve Optimization problem for a given gurvature sign sequence
    u_vec, z_vec, cost, status= solveMPC(car_size,n,m,Tmpc,z0,zT,zmin,zmax,umin,umax,obs,side)
    if status==:Optimal
        if cost<bestcost
            bestcost=cost
            print("new best index:",j)
            usol=u_vec
            zsol=z_vec
        end
    end
end
print("best_cost: ",bestcost)
#print(zsol)
#print(usol)




In [77]:
function simulate_car_MPC(Tmpc,z0,u,rep,L)
    Tsim=2*rep*Tmpc
    us = zeros(1,Tsim)
    zs = zeros(3,Tsim+1)
    t=1
    zs[:,1] = z0[:]
    for k = 0:Tmpc-1        
            for j = 1:rep        
                t=t+1
                zs[1,t] = zs[1,t-1] + 1/rep*u[1,k]*cos(zs[3,t-1])
                zs[2,t] = zs[2,t-1] + 1/rep*u[1,k]*sin(zs[3,t-1])
                zs[3,t] = zs[3,t-1] 
                us[t-1]=0
            end
            for l = 1:rep       
                t=t+1
                zs[1,t] = zs[1,t-1] -u[2,k]*sin(zs[3,t-1])+u[2,k]*sin(zs[3,t-1]+1/rep*u[3,k])
                zs[2,t] = zs[2,t-1] +u[2,k]*cos(zs[3,t-1])-u[2,k]*cos(zs[3,t-1]+1/rep*u[3,k])
                zs[3,t] = zs[3,t-1] + 1/rep*u[3,k]
            us[t-1]= atan(L/u[2,k])
            end
    end
    
    return  us, zs
end



simulate_car_MPC (generic function with 1 method)

###Next cell plots the parking manouver

In [78]:
@pyimport matplotlib.pyplot as plt

# Generate vectors for plots:
#print(zsol)
rep=6
Tsim=2*rep*Tmpc
u_vec,z_vec=simulate_car_MPC(Tmpc,z0,usol,rep,L)
#print(z_vec)

#Find Axis Limits
xmin = minimum([z_vec[1,:]])
xmax = maximum([z_vec[1,:]])
xmin, xmax = xmin - 0.1(xmax-xmin), xmax+ 0.1*(xmax-xmin)
ymin = minimum([z_vec[2,:]])
ymax = maximum([z_vec[2,:]])
ymin, ymax = ymin - 0.1(ymax-ymin), ymax+ 0.1*(ymax-ymin)

function plot_car(x,y,psi,beta,car_size,pt)
    l=car_size[1]
    dr=car_size[2]
    w=car_size[3]
    A=[x+l*cos(psi)+w*sin(psi),y+l*sin(psi)-w*cos(psi)]
    B=[x-dr*cos(psi)+w*sin(psi),y-dr*sin(psi)-w*cos(psi)]
    C=[x+l*cos(psi)-w*sin(psi),y+l*sin(psi)+w*cos(psi)]
    D=[x-dr*cos(psi)-w*sin(psi),y-dr*sin(psi)+w*cos(psi)]
    #wheels
    lw=0.4
    E=[A[1]+lw*cos(psi+beta),A[2]+lw*sin(psi+beta)]
    F=[C[1]+lw*cos(psi+beta),C[2]+lw*sin(psi+beta)]
    
    pt.plot([B[1] ,A[1]],[B[2], A[2]],"bo-")
    pt.plot([C[1] ,D[1]],[C[2], D[2]],"bo-")
    pt.plot([C[1] ,A[1]],[C[2], A[2]],"bo-")
    pt.plot([B[1] ,D[1]],[B[2], D[2]],"bo-")
    pt.plot([A[1] ,E[1]],[A[2], E[2]],"go-")
    pt.plot([C[1] ,F[1]],[C[2], F[2]],"go-")
end


#Construct Figure and Plot Data
fig = figure()
ax = plt.axes()
ax = plt.axes(xlim = (-20,20),ylim=(-2,20))
#ax = plt.axes(xlim = (xmin,xmax),ylim=(ymin,ymax))
plt.plot(transpose(z_vec[1,:]),transpose(z_vec[2,:]), "r-")

#Plot sow in TIME
for i=1:1:Tsim
    plot_car(z_vec[1,i],z_vec[2,i],z_vec[3,i],u_vec[1,i],car_size,plt)
end
plot_car(z_vec[1,[end]],z_vec[2,[end]],z_vec[3,[end]],u_vec[1,[end]],car_size,plt)



##end#Plot in SPACE
#NtP=6
#u_history = zeros(m,NtP*Tsim)
#z_history = zeros(n,NtP*Tsim)
#k=1
#for i=1:T
#    z_t=z_vec[:,i]
#    u_t=u_vec[:,i]
#    ddt=0.#5
 #   for j=0:ddt:dt
#        plot_car(z_t[1],z_t[2],z_t[3],u_t[1],l,width,plt)
#        z_history[:,k] = z_t[:]
 #       u_history[:,k] = u_t[:]
 #       z_t = z_t + ddt*zdot_fun(z_t,u_t,l)
 #       k=k+1
 #   en#d
#end
plt.plot([obs[1] ,obs[1]],[0, obs[3]],"r-")
plt.plot([obs[2] ,obs[2]],[0, obs[3]],"r-")
plt.plot([-20 ,obs[1]],[obs[3], obs[3]],"r-")
plt.plot([obs[2] ,20],[obs[3], obs[3]],"r-")
plt.plot([-20 ,20],[zmax[2]+1, zmax[2]+1],"r-")
plt.show()


#Plot start and end points
#ax[:plot](z_vec[1,1],z_vec[2,1], "ro")
#ax[:plot](z_vec[1,end],z_vec[2,end], "rs")

### Next makes a video of the manouver

In [64]:
#Pkg.add("PyPlot")
#Pkg.add("PyCall")
#Pkg.add("VideoIO")
using PyPlot
using PyCall
@pyimport matplotlib.animation as anim


pygui(true)

# First set up the figure, the axis, and the plot element we want to animate
fig = figure()
#ax = plt.axes(xlim = (xmin,xmax),ylim=(ymin,ymax))
#ax = plt.axes(xlim=(-10, 5), ylim=(-2, 10))
ax = plt.axes(xlim=(-15, 15), ylim=(-0, 20))
#ax2 = plt.axes()
global line1 = ax[:plot]([], [], "ro-")[1]
global line2 = ax[:plot]([], [], "ro-")[1]
global line3 = ax[:plot]([], [], "ro-")[1]
global line4 = ax[:plot]([], [], "ro-")[1]
global line5 = ax[:plot]([], [], "go-")[1]
global line6 = ax[:plot]([], [], "go-")[1]
ax[:plot]([obs[1] ,obs[1]],[0, obs[3]],"r-")
ax[:plot]([obs[2] ,obs[2]],[0, obs[3]],"r-")
ax[:plot]([-20 ,obs[1]],[obs[3], obs[3]],"r-")
ax[:plot]([obs[2] ,20],[obs[3], obs[3]],"r-")
ax[:plot]([-20 ,20],[zmax[2]+1, zmax[2]+1],"r-")
#plt.plot([-3 ,-3],[-3, 0],"r-")
#plt.plot([3 ,3],[-3, 0],"r-")
#plt.plot([-10 ,-3],[0, 0],"r-")
#plt.plot([3 ,15],[0, 0],"r-")

function points_car(x,y,psi,beta,car_size)
    l=car_size[1]
    dr=car_size[2]
    w=car_size[3]

    A=[x+l*cos(psi)+w*sin(psi),y+l*sin(psi)-w*cos(psi)]
    B=[x-dr*cos(psi)+w*sin(psi),y-dr*sin(psi)-w*cos(psi)]
    C=[x+l*cos(psi)-w*sin(psi),y+l*sin(psi)+w*cos(psi)]
    D=[x-dr*cos(psi)-w*sin(psi),y-dr*sin(psi)+w*cos(psi)]
    #wheels
    lw=0.6
    E=[A[1]+lw/2*cos(psi+beta),A[2]+lw/2*sin(psi+beta)]
    F=[C[1]+lw/2*cos(psi+beta),C[2]+lw/2*sin(psi+beta)]
    G=[A[1]-lw/2*cos(psi+beta),A[2]-lw/2*sin(psi+beta)]
    H=[C[1]-lw/2*cos(psi+beta),C[2]-lw/2*sin(psi+beta)]
    return A,B,C,D,E,F,G,H
end
 

# initialization function: plot the background of each frame
function init()
    global line1
    global line2
    global line3
    global line4
    global line5
    global line6
    global line7
    line1[:set_data]([], [])
    line2[:set_data]([], [])
    line3[:set_data]([], [])
    line4[:set_data]([], [])
    line5[:set_data]([], [])
    line6[:set_data]([], [])
    return (line1,line2,line3,line4,line5,line6,None)
end

# animation function.  This is called sequentially
function animate(i)
    k=i+1
    global line1
    global line2
    global line3
    global line4
    global line5
    global line6
    global line7
    if k<=Tsim+1
        A,B,C,D,E,F,G,H=points_car(z_vec[1,k],z_vec[2,k],z_vec[3,k],u_vec[k],car_size)
    else
        A,B,C,D,E,F,G,H=points_car(z_vec[1,end],z_vec[2,end],z_vec[3,end],u_vec[end],car_size)
    end
    line1[:set_data]([B[1] ,A[1]],[B[2], A[2]])
    line2[:set_data]([C[1] ,D[1]],[C[2], D[2]])
    line3[:set_data]([C[1] ,A[1]],[C[2], A[2]])
    line4[:set_data]([B[1] ,D[1]],[B[2], D[2]])
    line5[:set_data]([G[1] ,E[1]],[G[2], E[2]])
    line6[:set_data]([H[1] ,F[1]],[H[2], F[2]])
    return (line1,line2,line3,line4,line5,line6,None)
end


# call the animator.  blit=True means only re-draw the parts that have changed.
myanim = anim.FuncAnimation(fig, animate, init_func=init, frames=Tsim+20,  interval=100,)
#myanim[:save]("parking.mp4", writer="ffmpeg",  extra_args=["-vcodec", "libx264"])
#myanim[:save]("parking2.mp4", extra_args=["-vcodec", "libx264", "-pix_fmt", "yuv420p"])


PyObject <matplotlib.animation.FuncAnimation object at 0x0000000037689048>

    ret = func(*args, **kwargs)
  File "C:\Users\Francesco\Anaconda3\lib\site-packages\matplotlib\animation.py", line 925, in _step
    still_going = Animation._step(self, *args)
  File "C:\Users\Francesco\Anaconda3\lib\site-packages\matplotlib\animation.py", line 784, in _step
    self._draw_next_frame(framedata, self._blit)
  File "C:\Users\Francesco\Anaconda3\lib\site-packages\matplotlib\animation.py", line 803, in _draw_next_frame
    self._draw_frame(framedata)
  File "C:\Users\Francesco\Anaconda3\lib\site-packages\matplotlib\animation.py", line 1106, in _draw_frame
    self._drawn_artists = self._func(framedata, *self._args)
IndexError: Julia exception: BoundsError()
Traceback (most recent call last):
  File "C:\Users\Francesco\Anaconda3\lib\site-packages\matplotlib\backend_bases.py", line 1290, in _on_timer
    ret = func(*args, **kwargs)
  File "C:\Users\Francesco\Anaconda3\lib\site-packages\matplotlib\animation.py", line 925, in _step
    still_going = Animation._step(self, *a